In [1]:
# %pip install torch==2.2.2+cu121 torchvision==0.17.2+cu121 \
#             torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121

In [4]:
# %pip install transformers accelerate bitsandbytes

In [4]:
# %pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import time
import json
import pandas as pd

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## Загрузка данных

In [6]:
with open("TEST_prompts_v2.json", "r", encoding="utf-8") as f:
    loaded_samples = json.load(f)

In [8]:
DEFAULT_SYSTEM_PROMPT = "Ты — русскоязычный автоматический ассистент офтальмолога. Ты создаешь текстовые описания глазного дна по его параметрам."

## Saiga Llama

Первичная загрузка

In [2]:
# MODEL_NAME = "IlyaGusev/saiga_llama3_8b"

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.bfloat16,
#     device_map="auto"
# )
# model.eval()
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

2025-04-22 14:19:56.152187: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-22 14:19:57.124428: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|██████████| 4/4 [08:42<00:00, 130.58s/it]


Загрузка модели из локальной памяти

In [4]:
model_path = '/home/jupyter/datasphere/project/modelcache/models--IlyaGusev--saiga_llama3_8b/snapshots/5bb9917bdb85340549662ebb62c8e522037ff3f3'

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

2025-04-27 12:19:21.760086: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-27 12:19:22.705010: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|██████████| 4/4 [04:20<00:00, 65.08s/it]


In [6]:
MODEL_NAME = "IlyaGusev/saiga_llama3_8b"

In [7]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}

In [8]:
DEFAULT_SYSTEM_PROMPT = "Ты — русскоязычный автоматический ассистент офтальмолога. Ты создаешь текстовые описания глазного дна по его параметрам."

In [9]:
dataframe_elements = []
for sample in loaded_samples:
    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": DEFAULT_SYSTEM_PROMPT
    }, {
        "role": "user",
        "content": sample
    }], tokenize=False, add_generation_prompt=True)

    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}

    start_time = time.time()

    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    total_time = time.time() - start_time
    
    dataframe_elem = {"text":output, "time":total_time}
    dataframe_elements.append(dataframe_elem)

In [10]:
df = pd.DataFrame(dataframe_elements)
df.to_csv("Saiga_data_v2.csv")

## YandexGPT-5 Lite

Первичная загрузка

In [ ]:
# MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-instruct"

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.bfloat16,
#     device_map="auto"
# )
# model.eval()
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Загрузка модели из локальной памяти

In [ ]:
model_path = '/home/jupyter/datasphere/project/modelcache/models--yandex--YandexGPT-5-Lite-8B-instruct/snapshots/b556811768376b46c69caab60c4d1b69df9faaa1'

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

2025-04-27 11:36:53.845805: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-27 11:36:57.370592: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-instruct"

Также установим ограничение на max_new_tokens = 1536, как и Saiga. Чтобы при этом обойти дефотное ограничение на 20. 

In [22]:
tokenizer.decode(2)

'</s>'

In [ ]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1536
generation_config.pad_token_id = 2
generation_config

In [ ]:
DEFAULT_SYSTEM_PROMPT = "Ты — русскоязычный автоматический ассистент офтальмолога. Ты создаешь текстовые описания глазного дна по его параметрам."

In [24]:
dataframe_elements = []
for sample in loaded_samples:
    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": DEFAULT_SYSTEM_PROMPT
    }, {
        "role": "user",
        "content": sample
    }], tokenize=False, add_generation_prompt=True)

    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}

    start_time = time.time()

    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    total_time = time.time() - start_time
    
    dataframe_elem = {"text":output, "time":total_time}
    dataframe_elements.append(dataframe_elem)

In [11]:
df = pd.DataFrame(dataframe_elements)
df.to_csv("Yandex_data_v2.csv")

## BioMistral

Первичная загрузка

In [3]:
# MODEL_NAME = "BioMistral/BioMistral-7B"

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.bfloat16,
#     device_map="auto"
# )
# model.eval()
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

2025-04-23 11:04:47.528423: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-23 11:04:53.645176: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Загрузка модели из локальной памяти

In [ ]:
model_path = '/home/jupyter/datasphere/project/modelcache/models--BioMistral--BioMistral-7B'

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

In [12]:
MODEL_NAME = "BioMistral/BioMistral-7B"

Также установим ограничение на max_new_tokens = 1536, как и Saiga. Чтобы при этом обойти дефотное ограничение на 20. 

In [6]:
tokenizer.decode(2)

'</s>'

In [7]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1536
generation_config.pad_token_id = 2
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "pad_token_id": 2
}

In [8]:
DEFAULT_SYSTEM_PROMPT = "Ты — русскоязычный автоматический ассистент офтальмолога. Ты создаешь текстовые описания глазного дна по его параметрам."

In [13]:
dataframe_elements = []
for sample in loaded_samples:
    prompt = tokenizer.apply_chat_template([{
        "role": "user",
        "content": sample
    }], tokenize=False, add_generation_prompt=True)

    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}

    start_time = time.time()

    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    total_time = time.time() - start_time
    
    dataframe_elem = {"text":output, "time":total_time}
    dataframe_elements.append(dataframe_elem)

In [14]:
df = pd.DataFrame(dataframe_elements)
df.to_csv("BioMistal_data.csv")

## TableLLama

Первичная загрузка

In [4]:
# MODEL_NAME = "osunlp/TableLlama"

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.bfloat16,
#     device_map="auto"
# )
# model.eval()
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

2025-04-23 13:54:29.908913: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-23 13:54:35.761029: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|██████████| 2/2 [03:50<00:00, 115.45s/it]


Загрузка модели из локальной памяти

In [14]:
model_path = '/home/jupyter/datasphere/project/modelcache/models--osunlp--TableLlama/snapshots/b4bd8bac8b7570dcfa01ca3ef4f8fd0ffef957ed'

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

2025-04-27 20:35:28.411080: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-27 20:35:33.758901: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|██████████| 2/2 [03:51<00:00, 115.57s/it]


In [16]:
MODEL_NAME = "osunlp/TableLlama"

Также установим ограничение на max_new_tokens = 1536, как и Saiga. Чтобы при этом обойти дефотное ограничение на 20. 

In [17]:
tokenizer.decode(2)

'</s>'

In [18]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1536
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 1536,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}

In [19]:
DEFAULT_SYSTEM_PROMPT = "Ты — русскоязычный автоматический ассистент офтальмолога. Ты создаешь текстовые описания глазного дна по его параметрам."

In [20]:
with open("TEST_samples_v2.json", "r", encoding="utf-8") as f:
    json_samples = json.load(f)

In [23]:
dataframe_elements = []
for sample_dict in json_samples:
    # prompt = f"{DEFAULT_SYSTEM_PROMPT}\n {loaded_samples[i]}\n"
    json_text = f"""
    {{
    "ДЗН": {{
        "Цвет": "{sample_dict["od_color"]}",
        "Монотонность": "{sample_dict["od_monotone"]}",
        "Размер": "{sample_dict["od_size"]}",
        "Форма": "{sample_dict["od_shape"]}",
        "Границы": "{sample_dict["od_border"]}",
        "Экскавация": {{
            "Размер": {sample_dict["od_excavation_size"]},
            "Сектор": {sample_dict["od_excavation_location"]}
        }},
        "Э/Д": {sample_dict["od_excavation_ratio"]},
        "Сосудистый пучок": {sample_dict["od_vessels_location"]},
    }},
    "Сосуды": {{
        "Артерии": {{
            "Ход": {sample_dict["vessels_art_course"]},
            "Извитость": {sample_dict["vessels_art_turtuosity"]},
            "Бифуркация": {sample_dict["vessels_art_bifurcation"]},
            "Калибр": {sample_dict["vessels_art_caliber"]}
        }},
        "Вены": {{
            "Ход": {sample_dict["vessels_vein_course"]},
            "Извитость": {sample_dict["vessels_vein_turtuosity"]},
            "Бифуркация": {sample_dict["vessels_vein_bifurcation"]},
            "Калибр": {sample_dict["vessels_vein_caliber"]}
        }},
        "А/В индекс": {sample_dict["vessels_ratio"]},
    }},
    "Макула": {{
        "Макулярный рефлекс": {sample_dict["macula_macular_reflex"]},
        "Фовеальный рефлекс": {sample_dict["macula_foveal_reflex"]},
    }},
    }}
    """
    
    prompt = f"""### Instruction:
{DEFAULT_SYSTEM_PROMPT}

### Input:
{json_text}

### Question:
На основе приведённого JSON-документа, содержащего описание глазного дна, сгенерируй полное текстовое описание, придерживаясь стилистики и формата примера.

**Пример 1:**
ДЗН серый, монотонность не наблюдается, нормального размера, правильной формы, границы четкие. Экскавация нормального размера, расположена в центре, э/д=0.4. Сосудистый пучок расположен центрально.  
Артерии и Вены имеют нормальный ход, нормальную извитость, бифуркация в норме и калибр нормальный, соотношение А/В = 2/3.
Макулярный рефлекс отсутствует, фовеальный рефлекс нормальный.

**Пример 2:**
ДЗН серый, границы четкие, форма правильная, размер нормальный. Экскавация нормальная, в центре. Сосудистый пучок расположен центрально.  
Артерии: ход смещён наружу, извитость нормальная, бифуркация под острым углом, калибр нормальный.
Вены: ход нормальный, извитость нормальная, бифуркация нормальная, калибр суженный.
Соотношение А/В = 2/3.
Макулярный рефлекс отсутствует, фовеальный рефлекс нормальный.

**Твоя задача:**  
Создай полное текстовое описание глазного дна по JSON-документу, следуй телеграфному стилю и примеру выше.

### Response:"""
    
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}

    start_time = time.time()

    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    
    total_time = time.time() - start_time
    
    dataframe_elem = {"text":output, "time":total_time}
    dataframe_elements.append(dataframe_elem)

In [24]:
df = pd.DataFrame(dataframe_elements)
df.to_csv("TableLlama_data_v2.csv")

## Saiga Gemma 3 12b

In [3]:
MODEL_NAME = "IlyaGusev/saiga_gemma3_12b"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.eval()

2025-04-27 21:13:11.297243: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-27 21:13:14.868163: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|██████████| 5/5 [06:35<00:00, 79.04s/it]


Gemma3ForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(4096, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (self_attn): SiglipAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
            

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

GenerationConfig {
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "do_sample": true,
  "eos_token_id": [
    1,
    106
  ],
  "max_new_tokens": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.95
}



In [11]:
dataframe_elements = []
for sample in loaded_samples:
    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": DEFAULT_SYSTEM_PROMPT
    }, {
        "role": "user",
        "content": sample
    }], tokenize=False, add_generation_prompt=True)

    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}

    start_time = time.time()

    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    total_time = time.time() - start_time
    
    dataframe_elem = {"text":output, "time":total_time}
    dataframe_elements.append(dataframe_elem)

In [12]:
df = pd.DataFrame(dataframe_elements)
df.to_csv("Gemma3_12b_v2.csv")

## RuAdapt

In [ ]:
model_path = '/home/jupyter/datasphere/project/RuadaptQwen2.5'

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

2025-04-27 20:24:16.670457: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-27 20:24:17.635629: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|██████████| 4/4 [08:15<00:00, 123.85s/it]


**NB** В конфиге генерации не прописано максиальное количество токенов на выходе, без этого их будет всего 20. 

Поэтому необходимо прописать это явно в параметрах генерации.

Зададим значение 1536, как в конфиге у Сайги. Также изменим некоторые другие параметры, чтобы получать более строгие генерации.

In [ ]:
MODEL_NAME = "RefalMachine/RuadaptQwen2.5-7B-Lite-Beta"

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1536
generation_config.temperature = 0.2
generation_config.top_k = 30
generation_config.top_p = 0.9
generation_config

GenerationConfig {
  "do_sample": true,
  "eos_token_id": 145111,
  "max_new_tokens": 1536,
  "pad_token_id": 145109,
  "repetition_penalty": 1.05,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}

In [ ]:
DEFAULT_SYSTEM_PROMPT = "Ты — русскоязычный автоматический ассистент офтальмолога. Ты создаешь текстовые описания глазного дна по его параметрам."

In [ ]:
dataframe_elements = []
for sample in loaded_samples:
    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": DEFAULT_SYSTEM_PROMPT
    }, {
        "role": "user",
        "content": sample
    }], tokenize=False, add_generation_prompt=True)

    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}

    start_time = time.time()

    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    total_time = time.time() - start_time
    
    dataframe_elem = {"text":output, "time":total_time}
    dataframe_elements.append(dataframe_elem)

In [ ]:
df = pd.DataFrame(dataframe_elements)
df.to_csv("RuAdapt_data_v2.csv")

## T-lite

In [ ]:
MODEL_NAME = "t-tech/T-lite-it-1.0"
PATH = '/home/jupyter/datasphere/project/modelcache/models--t-tech--T-lite-it-1.0/snapshots/fbabc76f32140416dc5b0ceef392c7778eec1312'

tokenizer = AutoTokenizer.from_pretrained(PATH)
model = AutoModelForCausalLM.from_pretrained(
    PATH, 
    torch_dtype="auto",
    device_map="auto"
)

model.eval()

2025-04-27 20:03:58.390163: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-27 20:03:59.353016: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.38it/s]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151665, 3584, padding_idx=151643)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-0

In [ ]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1536
generation_config.temperature = 0.2
generation_config.top_k = 30
generation_config.top_p = 0.9
generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "max_new_tokens": 1536,
  "pad_token_id": 151643,
  "repetition_penalty": 1.05,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}

In [ ]:
DEFAULT_SYSTEM_PROMPT = "Ты — русскоязычный автоматический ассистент офтальмолога. Ты создаешь текстовые описания глазного дна по его параметрам."

In [ ]:
dataframe_elements = []
for sample in loaded_samples:
    prompt = tokenizer.apply_chat_template([{
        "role": "system",
        "content": DEFAULT_SYSTEM_PROMPT
    }, {
        "role": "user",
        "content": sample
    }], tokenize=False, add_generation_prompt=True)

    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}

    start_time = time.time()

    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    total_time = time.time() - start_time
    
    dataframe_elem = {"text":output, "time":total_time}
    dataframe_elements.append(dataframe_elem)

In [ ]:
df = pd.DataFrame(dataframe_elements)
df.to_csv("T_lite_data_v2.csv")

## Gemma 3

In [ ]:
# %pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [ ]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from PIL import Image
import requests
import torch

In [ ]:
model_id = "google/gemma-3-4b-it"
token = '<hf–token>'

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto", token=token
).eval()

Loading checkpoint shards: 100%|██████████| 2/2 [04:40<00:00, 140.15s/it]


In [ ]:
processor = AutoProcessor.from_pretrained(model_id, token=token)

In [ ]:
DEFAULT_SYSTEM_PROMPT = "Ты — русскоязычный автоматический ассистент офтальмолога. Ты создаешь текстовые описания глазного дна по его параметрам."

In [ ]:
dataframe_elements = []
for sample in loaded_samples:
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": DEFAULT_SYSTEM_PROMPT}]
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": loaded_samples[0]}
            ]
        }
    ]

    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)

    input_len = inputs["input_ids"].shape[-1]
    
    start_time = time.time()
    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
        generation = generation[0][input_len:]

    decoded = processor.decode(generation, skip_special_tokens=True)
    total_time = time.time() - start_time
    
    dataframe_elem = {"text":decoded, "time":total_time}
    dataframe_elements.append(dataframe_elem)

In [ ]:
df = pd.DataFrame(dataframe_elements)
df.to_csv("Gemma3_4b_v2.csv")